Whiskas cat food, shown above, is manufactured by Uncle Ben’s. Uncle Ben’s want to produce their cat food products as cheaply as possible while ensuring they meet the stated nutritional analysis requirements shown on the cans. Thus they want to vary the quantities of each ingredient used (the main ingredients being chicken, beef, mutton, rice, wheat and gel) while still meeting their nutritional standards.

The costs of the chicken, beef, and mutton are \$0.013, \$0.008 and \$0.010 respectively, while the costs of the rice, wheat and gel are \$0.002, \$0.005 and $0.001 respectively. (All costs are per gram.) For this exercise we will ignore the vitamin and mineral ingredients. (Any costs for these are likely to be very small anyway.)

Each ingredient contributes to the total weight of protein, fat, fibre and salt in the final product. The contributions (in grams) per gram of ingredient are given in the table below.


|Stuff	|Costs | Protein	|Fat	|Fibre	|Salt|
|---|---|---|---|---|---|
|Chicken	|0.013 | 0.100	|0.080	|0.001	|0.002|
|Beef	|0.008 | 0.200	|0.100	|0.005	|0.005|
|Rice	|0.002| 0.000	|0.010	|0.100	|0.002|
|Wheat bran	|0.005 | 0.040	|0.010	|0.150	|0.008|
| Mutton | 0.010| 0.15 | 0.110 | 0.003 | 0.007 |
| Gel | 0.001| 0.010 | 0.007 | 0.02 | 0.01 |

In [ ]:
"""
The Full Whiskas Model Python Formulation for the PuLP Modeller

Authors: Antony Phillips, Dr Stuart Mitchell  2007
"""

import pandas as pd
from pandas import DataFrame


# the list of ingredients Creates a list of the Ingredients
ingredients = ['chicken', 'beef', 'mutton', 'rice', 'wheat', 'gel']

# data is stored in a pandas dataframe, indexed by the ingredients
pdf = DataFrame({"cost":    [   13,      8,   10,     2,     5,    1],
                 "protein": [0.100, 0.200, 0.150, 0.000, 0.040,    0],
                 "fat":     [0.080, 0.100, 0.110, 0.010, 0.010,    0],
                 "fiber":   [0.001, 0.005, 0.003, 0.100, 0.150,    0],
                 "salt":    [0.002, 0.005, 0.007, 0.002, 0.008, 0.003]},
                index=ingredients)

# print the dataframe as a table
pdf

In [ ]:
try:
    import docplex.mp
except:
    !pip install docplex

In [ ]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

In [ ]:
if env.has_matplotlib:
    %matplotlib inline
    import matplotlib.pyplot as plt

def display(values, labels, title):
    if env.has_matplotlib:
        N = len(values)

        ind = range(N)  # the x locations for the groups
        width = 0.35       # the width of the bars

        fig, ax = plt.subplots()
        rects1 = ax.bar(ind, values, width, color='r')	
        ax.set_title(title)
        ax.set_xticks([ind[i] +width/2 for i in ind])
        ax.set_xticklabels( labels )	

        plt.show()
    else:
        print("warning: no display")

In [ ]:
display(pdf["cost"], ingredients, "costs")

In [ ]:
from docplex.mp.context import Context
context = Context(url="PUT_YOUR_DOCLOUD_URL_HERE", 
                  api_key="PUT_YOUR_DOCLOUD_KEY_HERE")

In [ ]:
from docplex.mp.model import Model

In [ ]:
# Create the 'prob' variable to contain the problem data
mdl = Model("The Whiskas Problem", context=context, sort_expressions=True)

A dictionary called ingredient_vars is created which contains the variables with bounds [0..100]. The reference keys to the dictionary are the ingredient names.

In [ ]:
# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
# Here, variables are continuous, but you might change this to integer variables to state
# that percentage values must be integers
ingredient_vars = mdl.continuous_var_dict(keys=ingredients, lb=0.0, ub=100)

Expressing constraints is straightforward using Python's arithmetic operators and pandas structures.

In [ ]:
# The objective function is added to 'prob' first
mdl.minimize(mdl.sum(ingredient_vars[i]*pdf.ix[i]["cost"] for i in ingredients))

# all ingredients sum up to 100 %
mdl.add_constraint(mdl.sum(ingredient_vars[i] for i in ingredients) == 100)
# max protein
mdl.add_constraint(mdl.sum(ingredient_vars[i] * pdf.ix[i]["protein"] for i in ingredients) >= 8.0)
mdl.add_constraint(mdl.sum(ingredient_vars[i] * pdf.ix[i]["fat"] for i in ingredients) >= 6.0)
mdl.add_constraint(mdl.sum(ingredient_vars[i] * pdf.ix[i]["fiber"] for i in ingredients) <= 2.0)
mdl.add_constraint(mdl.sum(ingredient_vars[i] * pdf.ix[i]["salt"] for i in ingredients) <= 0.4)
mdl.print_information()

Now let's solve the model, when successful, solve() returns a SolveSolution object, else it returns None.

In [ ]:
sol = mdl.solve()
if sol is None:
    print("Problem has no solution")
else:
    print("Problem solved successfully")

The optimised objective function value is printed to the screen

In [ ]:
print("Total Cost of Ingredients per can = {0:.4f}".format(mdl.objective_value))

The solution object as returned by solve() can be printed to show the values of variables
in the optimal solution. All variables with value zero are not printed.

In [ ]:
print(sol)

As the variables represent percentages, we display solution values in a pie chart. This code requires the **matplotlib** module.

In [ ]:
if env.has_matplotlib:
    from pylab import *

    # make a square figure and axes
    figure(1, figsize=(6,6))
    ax = axes([0.1, 0.1, 0.8, 0.8])

    # we collect only nonzero percentage to avoid cluttering the diagram
    labels = []
    fracs = []
    for i in ingredients:
        frac = ingredient_vars[i].solution_value
        if frac != 0:
            labels.append(i)
            fracs.append(frac)
            
    pie(fracs, labels=labels,
        autopct='%1.1f%%', shadow=False, startangle=90,
        colors = ['Tan', 'LightGreen', 'LightBlue', "plum"])
                    # The default startangle is 0, which would start
                    # the Frogs slice on the x-axis.  With startangle=90,
                    # everything is rotated counter-clockwise by 90 degrees,
                    # so the plotting starts on the positive y-axis.
    mytitle = "Mix to get a Can for a cost of {:.4f}".format(mdl.objective_value)
    title(mytitle, bbox={'facecolor':'0.8', 'pad':5})

    show()

After solve, we can insert the percentage found in the pandas frame:

In [ ]:
pdf["percentage"] = [float(ingredient_vars[i]) for i in ingredients]

# print the updated dataframe with percentage solutions
pdf